# **Analiza danych ankietowych - Sprawozdanie I**

## *Autorzy: Julia Krempińska, Filip Miśkiewicz*

## **Biblioteki**

In [ ]:
import numpy as np
import scipy.stats as st
import pandas as pd
import fisher
import warnings

warnings.filterwarnings("ignore")
%matplotlib inline 

## **Przygotowanie danych**

In [13]:
# Wczytanie danych z pliku ankieta.csv
df = pd.read_csv("ankieta.csv", sep=";")

In [14]:
WIEK_KAT = np.zeros(len(df["WIEK"]))
for i in range(len(df["WIEK"])): #sprawdzanie warunków
    if 35 < df["WIEK"][i] <= 45:
        WIEK_KAT[i] = 1
    if 45 < df["WIEK"][i] <= 55:
        WIEK_KAT[i] = 2
    if df["WIEK"][i] > 55:
        WIEK_KAT[i] = 3
df["WIEK_KAT"] = WIEK_KAT
df.tail() #0 jeśli < 35

,DZIAŁ,STAŻ,CZY_KIER,PYT_1,PYT_2,PYT_3,PŁEĆ,WIEK,WIEK_KAT
195,HR,2,Nie,1,2,2,M,42,1.0
196,HR,2,Nie,1,-1,-1,K,35,0.0
197,HR,2,Nie,-1,-2,-2,K,39,1.0
198,HR,2,Nie,1,2,1,K,48,2.0
199,HR,2,Nie,1,2,2,K,42,1.0


## **Część I**

### **Zadanie 1**

In [5]:
n = 200

counts = np.array([14, 17, 40, 100, 29])
categories = [
    "Bardzo niezadowolony", 
    "Niezadowolony", 
    "Bez zdania", 
    "Zadowolony", 
    "Bardzo zadowolony"
]

# Estymatory p̂
p_hat = counts / n

# Parametry dla CI
alpha = 0.05
z = st.norm.ppf(1 - alpha / 2)

# Liczenie błędu standardowego bezpiecznie
standard_error = np.sqrt(np.clip(p_hat * (1 - p_hat) / n, 0, None))

# Przedziały ufności
ci_lower = p_hat - z * standard_error
ci_upper = p_hat + z * standard_error

# Ograniczenie do [0, 1]
ci_lower = np.clip(ci_lower, 0, 1)
ci_upper = np.clip(ci_upper, 0, 1)

# Tabela wyników
df = pd.DataFrame({
    "Kategoria": categories,
    "Liczba odpowiedzi": counts,
    "Estymator p̂": p_hat,
    "95% CI dolny": ci_lower,
    "95% CI górny": ci_upper
})

df = df.round(4)

display(df)


,Kategoria,Liczba odpowiedzi,Estymator p̂,95% CI dolny,95% CI górny
0,Bardzo niezadowolony,14,0.070,0.0346,0.1054
1,Niezadowolony,17,0.085,0.0463,0.1237
2,Bez zdania,40,0.200,0.1446,0.2554
3,Zadowolony,100,0.500,0.4307,0.5693
4,Bardzo zadowolony,29,0.145,0.0962,0.1938


### **Zadanie 2**

In [6]:
def chi_squared_test(x, p0, alpha=0.05):
    """
    Test chi-kwadrat Pearsona i największej wiarygodności dla hipotezy H0: p = p0
    
    Parametry:
    x     - wektor zliczeń (np. [14, 17, 40, 100, 29])
    p0    - wektor p0 z hipotezy zerowej (np. [0.1, 0.1, 0.2, 0.4, 0.2])
    alpha - poziom istotności (np. 0.05)
    
    Zwraca:
    Słownik z wynikami testów: statystyki, p-value, decyzja o odrzuceniu H0
    """
    x = np.array(x)
    p0 = np.array(p0)
    n = np.sum(x)
    k = len(x)
    
    # Oczekiwane wartości
    expected = n * p0

    # Statystyka testu Pearsona
    with np.errstate(divide='ignore', invalid='ignore'):
        T_Pearson = np.nansum((x - expected)**2 / expected)

    # Statystyka testu największej wiarygodności (log-likelihood ratio)
    ratio = np.where(x == 0, 1, x / expected)
    T_LR = 2 * np.nansum(np.where(x == 0, 0, x * np.log(ratio)))

    # Stopnie swobody
    df = k - 1

    # Wartości krytyczne
    critical_value = st.chi2.ppf(1 - alpha, df)

    # P-value
    pval_pearson = 1 - st.chi2.cdf(T_Pearson, df)
    pval_lr = 1 - st.chi2.cdf(T_LR, df)

    # Decyzja
    decision_pearson = "Odrzucamy H0" if pval_pearson < alpha else "Brak podstaw do odrzucenia H0"
    decision_lr = "Odrzucamy H0" if pval_lr < alpha else "Brak podstaw do odrzucenia H0"

    return {
        "Test Pearsona": {
            "Statystyka": T_Pearson,
            "p-value": pval_pearson,
            "Wartość krytyczna": critical_value,
            "Decyzja": decision_pearson
        },
        "Test największej wiarygodności": {
            "Statystyka": T_LR,
            "p-value": pval_lr,
            "Wartość krytyczna": critical_value,
            "Decyzja": decision_lr
        },
        "Stopnie swobody": df,
        "Poziom istotności": alpha
    }


### **Zadanie 3**

In [ ]:
counts = (df['PYT_1'].value_counts())
print(counts)
     
p0 = np.full(len(counts), 1 / len(counts))  
print(p0)

T_Pearson= chi_squared_test(counts, p0, alpha=0.05)

display("Statystyka chi-kwadrat Pearsona:", T_Pearson)

PYT_1
 1    100
 0     40
 2     29
-1     17
-2     14
Name: count, dtype: int64
[0.2 0.2 0.2 0.2 0.2]


'Statystyka chi-kwadrat Pearsona:'

{'Test Pearsona': {'Statystyka': 123.15,
  'p-value': 0.0,
  'Wartość krytyczna': 9.487729036781154,
  'Decyzja': 'Odrzucamy H0'},
 'Test największej wiarygodności': {'Statystyka': 106.11862894751276,
  'p-value': 0.0,
  'Wartość krytyczna': 9.487729036781154,
  'Decyzja': 'Odrzucamy H0'},
 'Stopnie swobody': 4,
 'Poziom istotności': 0.05}

## **Część II**

### **Zadanie 4**

Do wykonania testu Fishera oraz testu Freemana-Haltona użyjemy funkcji z bibliotek *fisher* oraz *scipy.stats*

### **Zadanie 5**

In [8]:
df2 = pd.crosstab(df['PŁEĆ'], df['CZY_KIER'])
display(df2)

odds_ratio, p_value = st.fisher_exact(df2, alternative='two-sided')
    
decision = "Odrzucamy H₀ – zmienne są zależne" if p_value < 0.05 else "Brak podstaw do odrzucenia H₀ - zmienne są niezależne"

display({
    "p-value": p_value,
    "Decyzja": decision
})

CZY_KIER,Nie,Tak
PŁEĆ,,
K,63,8
M,110,19


{'p-value': 0.6659028889666552,
 'Decyzja': 'Brak podstaw do odrzucenia H₀ - zmienne są niezależne'}

### **Zadanie 6**

In [31]:
def fisher_monte_carlo_from_table(data1, data2, B=10000, random_state=None):
    data_frame = pd.crosstab(df[data1], df[data2])

    np.random.seed(random_state)

    # Oryginalna statystyka chi-kwadrat
    chi2_obs, _, _, _ = st.chi2_contingency(data_frame, correction=False)

    # Rekonstrukcja "płaskiego" dataframe do permutacji
    expanded_df = df[[data1, data2]]
    
    shuffled = np.random.permutation(expanded_df[data2])
    simulated_table = pd.crosstab(expanded_df[data1], shuffled)

    # Symulacje
    count = 0
    for _ in range(B):
        shuffled = np.random.permutation(expanded_df[data2])
        simulated_table = pd.crosstab(expanded_df[data1], shuffled)
        
        # Dopasuj do kształtu oryginału (w razie braków)
        simulated_table = simulated_table.reindex(index=data_frame.index, columns=data_frame.columns, fill_value=0)
        
        chi2_sim, _, _, _ = st.chi2_contingency(simulated_table, correction=False)
        
        if chi2_sim >= chi2_obs:
            count += 1

    p_value = count / B
    return chi2_obs, p_value

In [ ]:
def check_pvalue(alpha=0.05):
    if p_value < alpha:
        print("\nWniosek: Odrzucamy hipotezę zerową (jest zależność).")
    else:
        print("\nWniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).")

In [35]:
alpha=0.05

a) zajmowanie stanowiska kierowniczego nie zależy od wieku 

In [37]:
# Przykład użycia:
chi2_obs, p_value = fisher_monte_carlo_from_table('WIEK_KAT', 'CZY_KIER', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 1.0803155311825832
P-wartość (Monte Carlo) = 0.827

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).


b) zajmowanie stanowiska kierowniczego nie zależy od stażu pracy

In [38]:
chi2_obs, p_value = fisher_monte_carlo_from_table('STAŻ', 'CZY_KIER', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 23.179908824020515
P-wartość (Monte Carlo) = 0.0

Wniosek: Odrzucamy hipotezę zerową (jest zależność).


c) stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb
w pierwszym badanym okresie nie zależy od zajmowanego stanowiska

In [39]:
chi2_obs, p_value = fisher_monte_carlo_from_table('PYT_2', 'CZY_KIER', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 13.11369172640849
P-wartość (Monte Carlo) = 0.024

Wniosek: Odrzucamy hipotezę zerową (jest zależność).


d) stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w
pierwszym badanym okresie nie zależy od stażu 

In [40]:
chi2_obs, p_value = fisher_monte_carlo_from_table('PYT_2', 'STAŻ', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 26.1193138645
P-wartość (Monte Carlo) = 0.0

Wniosek: Odrzucamy hipotezę zerową (jest zależność).


e) stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w
pierwszym badanym okresie nie zależy od płci

In [41]:
chi2_obs, p_value = fisher_monte_carlo_from_table('PYT_2', 'PŁEĆ', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 2.2700823224897917
P-wartość (Monte Carlo) = 0.548

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).


f) stopień zadowolenia ze szkoleń w kontekście dopasowania do indywidualnych potrzeb w
pierwszym badanym okresie nie zależy od wieku

In [42]:
chi2_obs, p_value = fisher_monte_carlo_from_table('PYT_2', 'WIEK_KAT', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 9.845826003518312
P-wartość (Monte Carlo) = 0.323

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).


In [43]:
CZY_ZADOW = np.zeros(len(df["PYT_2"]))
for i in range(len(df["PYT_2"])):
    CZY_ZADOW[i] = np.sign(df["PYT_2"][i]) #pozytywna odpowiedź > 0, negatywna < 0
df["CZY_ZADOW"] = CZY_ZADOW
df.tail()

,DZIAŁ,STAŻ,CZY_KIER,PYT_1,PYT_2,PYT_3,PŁEĆ,WIEK,WIEK_KAT,CZY_ZADOW
195,HR,2,Nie,1,2,2,M,42,1.0,1.0
196,HR,2,Nie,1,-1,-1,K,35,0.0,-1.0
197,HR,2,Nie,-1,-2,-2,K,39,1.0,-1.0
198,HR,2,Nie,1,2,1,K,48,2.0,1.0
199,HR,2,Nie,1,2,2,K,42,1.0,1.0


In [44]:
chi2_obs, p_value = fisher_monte_carlo_from_table('CZY_ZADOW', 'CZY_KIER', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 0.08183603554685095
P-wartość (Monte Carlo) = 0.825

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).


In [45]:
chi2_obs, p_value = fisher_monte_carlo_from_table('CZY_ZADOW', 'STAŻ', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 1.7773575924124572
P-wartość (Monte Carlo) = 0.442

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).


In [46]:
chi2_obs, p_value = fisher_monte_carlo_from_table('CZY_ZADOW', 'PŁEĆ', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)

Chi² = 0.23290747005849552
P-wartość (Monte Carlo) = 0.658

Wniosek: Brak podstaw do odrzucenia hipotezy zerowej (brak zależności).


In [ ]:
chi2_obs, p_value = fisher_monte_carlo_from_table('CZY_ZADOW', 'WIEK_KAT', B=1000)
print(f"Chi² = {chi2_obs}")
print(f"P-wartość (Monte Carlo) = {p_value}")

check_pvalue(alpha)